### Step 1 - Importing required Libraries

In [ ]:
%matplotlib inline
import os
import pandas as pd
import csv
import matplotlib.pyplot as plt
import pydicom
import cv2

### Step 2 - Addressing the path of images

In [ ]:
folder_path = "./images"
images = os.listdir(folder_path)
# print(images); # this print command is to be sure that all the files in the folder are in .dcm format

### Step 3 - Loading description from a csv file

It is possible to add any available description in dicom_image_description.csv file

In [ ]:
dicom_image_description = pd.read_csv("dicom_image_description.csv")
# print(dicom_image_description)

### Step 4 - Writing dcm files data into a csv file
#### After run, in the working directory, find the 'Patient_Detail.csv' 

In [ ]:
with open('Patient_Detail.csv', 'w', newline ='') as csvfile:
    fieldnames = list(dicom_image_description["Description"])
    writer = csv.writer(csvfile, delimiter=',')
    writer.writerow(fieldnames)
    for n, image in enumerate(images):
        dataset = pydicom.dcmread(os.path.join(folder_path, image))
        rows = []
        for field in fieldnames:
            if dataset.get(field, None) is None:
                rows.append('')
            else:
                x = str(dataset.data_element(field)).replace("'", "")
                y = x.find(":")
                x = x[y+2:]
                rows.append(x)
        writer.writerow(rows)

### Step 5 - Convert all dcm files into jpg format, at once


In [ ]:
# make it True if you want in PNG format
PNG = False

jpg_folder_path = "images_jpg"
for n, image in enumerate(images):
    ds = pydicom.dcmread(os.path.join(folder_path, image))
    pixel_array_numpy = ds.pixel_array
    if PNG == False:
        image = image.replace('.dcm', '.jpg')
    else:
        image = image.replace('.dcm', '.png')
    cv2.imwrite(os.path.join(jpg_folder_path, image), pixel_array_numpy)
    if n % 50 == 0:
        print('{} image converted'.format(n))


### Step 6 - Preview a DICOM image in python

In [ ]:
# as an example I put 000000.dcm file in ./images directory in this working dierctory.
image_path = './images/000000.dcm'
image_data = pydicom.dcmread(image_path)
plt.imshow( image_data.pixel_array)

plt.show()

### Step 7 - Plotting all dcm images along some of their useful information

In [ ]:
for n, image in enumerate(images):
    dataset = pydicom.dcmread(os.path.join(folder_path, image))
    pat_name = dataset.PatientName
    display_name = pat_name.family_name + ", " + pat_name.given_name
    print("Patient's name...:", display_name)
    print("Patient id.......:", dataset.PatientID)
    print("Modality.........:", dataset.Modality)
    print("Study Date.......:", dataset.StudyDate)
    if 'PixelData' in dataset:
        rows = int(dataset.Rows)
        cols = int(dataset.Columns)
        print("Image size.......: {rows:d} x {cols:d}, {size:d} bytes".format(
            rows=rows, cols=cols, size=len(dataset.PixelData)))
        if 'PixelSpacing' in dataset:
            print("Pixel spacing....:", dataset.PixelSpacing)
    # use .get() if not sure the item exists, and want a default value if missing
    print("Slice location...:", dataset.get('SliceLocation', "(missing)"))
    print("----------------")
    # plot the image using matplotlib
    plt.imshow(dataset.pixel_array)
    plt.show()